<!-- :Author: Arthur Goldberg <Arthur.Goldberg@mssm.edu> -->
<!-- :Date: 2020-07-13 -->
<!-- :Copyright: 2020, Karr Lab -->
<!-- :License: MIT -->
# DE-Sim tutorial

DE-Sim makes it easy to build and simulate discrete-event models.
This page introduces the basic concepts of discrete-event modeling and teaches you how to build and simulate discrete-event model with DE-Sim. 

## Installation
Use `pip` to install `de_sim`.

`pip install de_sim`

## Minimal DE-Sim model and simulation
An Object-oriented (OO) discrete-event simulation (DES) can be built and run in DE-Sim in three steps: define a message type; define a simulation class; and build and run a simulation.

### 1: Create event message types by subclassing `SimulationMessage`, a customized DE-Sim class.

As the name suggests, DES models execute events at discrete instants of time.
Each event contains an event message that provides data needed by a simulation object that executes the event.
For some events, the type of the event message provides all the information required to execute the event, as illustrated by event message class `MessageSentToSelf`. For other events, the event message contains data in one or more attributes, as illustrated by `MessageWithAttribute`.

Event message classes must be documented by docstrings.

In [4]:
from de_sim.simulation_engine import SimulationEngine
from de_sim.simulation_message import SimulationMessage
from de_sim.simulation_object import ApplicationSimulationObject

class MessageSentToSelf(SimulationMessage):
    "A message type with no attributes"

class MessageWithAttribute(SimulationMessage):
    "An event message type with an attribute called 'value'"
    attributes = ['value']

### 2: Define simulation application object types by subclassing `ApplicationSimulationObject`.

As in an OO program, the classes used by a DES application encapsulate data and code that determine the application's behavior.
Subclasses of `ApplicationSimulationObject`, called *simulation objects*, are special simulation classes that handle simulation events.
Simulation objects are like threads, as they are run by a simulation's scheduler and suspend execution when they have no work to do. 
But DES simulation objects and threads are scheduled by different algorithms.
Whereas threads are scheduled whenever they have work to do,
a DES scheduler schedules simulation objects to ensure that events occur in simulation time order:

1. All events in a simulation are executed in non-decreasing time order. (Events with equal simulation times are scheduled according to the tie-breaking rules discussed below.) 

By guaranteeing this behavior, a DES scheduler ensures that causality relationships between events are respected.
This rule has two consequences:

1. Each simulation object executes its events in increasing time order. DE-Sim achieves this by having any simulation object that receives multiple events at a given simulation time execute *all* of the events together.
2. All synchronization between simulation objects is controlled by the simulation times of events.

Below, we define a simulation class called `SimpleSimulationObject` that illustrates many of the key features of DE-Sim's `ApplicationSimulationObject`.

In [5]:
class SimpleSimulationObject(ApplicationSimulationObject):
    """ A simple example DE-Sim simulation object type

    Attributes:
        name: The name of an instance of this object; each instance must have a unique name.
        delay: A float, which provides the delay between events.
    """

    def __init__(self, name, delay):
        """ Initialize a simulation object

        Args:
            name: The unique name of this instance of this object.
            delay: The delay between events.
        """
        super().__init__(name)
        self.delay = delay

    def init_before_run(self):
        """ Initialize before a simulation run. Send the initial events. Called by the simulator.
        """
        self.send_event(self.delay, self, MessageSentToSelf())

    def handle_simulation_event(self, event):
        """ Handle a simulation event

        Args:
            event: The DE-Sim event being executed. Not used in this example.
        """
        self.send_event(self.delay, self, MessageSentToSelf())

    # event_handlers is a list of pairs that maps each event message type
    # received by this simulation object type to the method that handles
    # the event message type
    event_handlers = [(MessageSentToSelf, handle_simulation_event)]

    # messages_sent registers all message types sent by this object
    messages_sent = [MessageSentToSelf]

A subclass of `ApplicationSimulationObject` contains special methods and attributes that define its simulation behavior.

* Special methods
  1. `init_before_run` (optional): just before a simulation run, after all objects have been loaded into a simulation, the simulator calls `init_before_run` in each simulation object. Simulation objects can send initial events and perform any other initialization in this method. A simulation must send at least one initial event to initiate the simulation's execution.
  2. event handlers: an event handler is a method that handles a simulation event. Event handlers have the signature `event_handler(event)`, where `event` is a DE-Sim simulation event (`de_sim.event.Event`). A subclass of `ApplicationSimulationObject` must define at least one event handler, like `handle_simulation_event` in the example above.
* Special attributes
  1. `event_handlers`: the attribute `event_handlers` must contain a list of pairs that maps each event message type received by a subclass of `ApplicationSimulationObject` to the subclass' event handler which handles the event message type. In the example above, `event_handlers` associates `MessageSentToSelf` event messages with the `handle_simulation_event` event handler. The object dispatch algorithm in the DE-Sim simulator scheduler executes an event by using the receiving object identified in the message and its `event_handlers` attribute to determine the object's method that should execute the event. It then dispatches execution to that method in the receiving object while passing the event as an argument.
  2. `messages_sent`: the types of messages sent by a subclass of `ApplicationSimulationObject` are listed in `messages_sent`, which is used to ensure that a simulation object doesn't send messages of the wrong type.

To schedule events, `ApplicationSimulationObject` provides the method

    send_event(delay, receiving_object, event_message)
    
which schedules an event to occur `delay` time units in the future at simulation object `receiving_object`, which will execute a simulation event containing `event_message`.
An event can be scheduled for any simulation object in a simulation, including the object scheduling the event, as shown in the example above.
Object-oriented DES terminology also describes the event message as being sent by the sending object at the message's send time (the simulation time when the event is scheduled) and being received by the receiving object at the event's receive time (the simulation time when the event is executed).

`event_message` must be an instance of a `SimulationMessage`, and may have attributes that contain data used by the event.
The event will be executed by an event handler in simulation object `receiving_object`,
with a parameter that is set to a simulation event containing `event_message` at its scheduled simulation time.
The example above illustrates this in its

    handle_simulation_event(self, event)

method.
In this example all simulation events are scheduled to be executed by the object that creates the event, but realistic simulations contain multiple simulation objects which schedule events for each other.

### 3: Execute a simulation application by creating a `SimulationEngine`, instantiating and adding the application's simulation objects, and running the simulation.

In [6]:
# create a simulation engine
simulation_engine = SimulationEngine()

# create a simulation object and add it to the simulation
simulation_engine.add_object(SimpleSimulationObject('object_1', 6))

# initialize the simulation, and send initial event messages
simulation_engine.initialize()
# run the simulation for 100 time units
num_events = simulation_engine.run(100).num_events
print('Executed', num_events, 'events')

Executed 16 events


## DE-Sim example with multiple object instances

This section presents an implementation of the parallel hold (PHOLD) model, which is frequently used to benchmark parallel DES (PDES) simulators.
We implement PHOLD as a DE-Sim model and use it to illustrate these features:

* Run multiple instances of a simulation object type
* Use multiple `SimulationMessage` types
* Run a stochastic simulation
* Record a simulation's predictions
* Use the `de_sim.event.Event` object passed to event handler methods
* Use `self.time` to access the current simulation time

In [7]:
""" Parallel hold (PHOLD) model commonly used to benchmark parallel discrete-event simulators :cite:`fujimoto1990performance`.

:Author: Arthur Goldberg <Arthur.Goldberg@mssm.edu>
:Date: 2016-06-10
:Copyright: 2016-2020, Karr Lab
:License: MIT
"""
import random

class MessageSentToSelf(SimulationMessage):
    "A message that's sent to self"

class MessageSentToOtherObject(SimulationMessage):
    "A message that's sent to another PHold simulation object"

class InitMsg(SimulationMessage):
    'initialization message'

MESSAGE_TYPES = [MessageSentToSelf, MessageSentToOtherObject, InitMsg]


class PholdSimulationObject(ApplicationSimulationObject):
    """ Run a PHOLD simulation

    Attributes:
        args: a :obj:`Namespace` that defines:
            `num_phold_objects`: the number of PHOLD objects to run
            `frac_self_events`: the fraction of events sent to `self`
            `time_max`: the end time for the simulation
    """
    def __init__(self, name, args):
        self.args = args
        super().__init__(name)

    def init_before_run(self):
        self.send_event(random.expovariate(1.0), self, InitMsg())

    @staticmethod
    def record_event_header():
        print('\t'.join(('Sender',
                         'Send',
                         "Receivr",
                         'Event',
                         'Message type')))
        print('\t'.join(('', 'time', '', 'time', '')))
        
    def record_event(self, event):
        record_format = '{}\t{:.2f}\t{}\t{:.2f}\t{}'
        print(record_format.format(event.sending_object.name,
                                   event.creation_time,
                                   event.receiving_object.name,
                                   self.time,
                                   type(event.message).__name__))

    def handle_simulation_event(self, event):
        """ Handle a simulation event """
        # Record this event
        self.record_event(event)
        # Schedule an event
        if random.random() < self.args.frac_self_events or \
            self.args.num_phold_objects == 1:
            receiver = self
        else:
            # Send the event to another randomly selected object
            # Pick an object index in [0, num_phold-2], and increment if self or greater
            obj_index = random.randrange(self.args.num_phold_objects - 1)
            if int(self.name) <= obj_index:
                obj_index += 1
            receiver = self.simulator.simulation_objects[str(obj_index)]

        if receiver == self:
            message_type = MessageSentToSelf
        else:
            message_type = MessageSentToOtherObject
        self.send_event(random.expovariate(1.0), receiver, message_type())

    event_handlers = [(sim_msg_type, 'handle_simulation_event') \
                      for sim_msg_type in MESSAGE_TYPES]

    # register the message types sent
    messages_sent = MESSAGE_TYPES


def create_and_run(args):

    # create a simulator
    simulator = SimulationEngine()

    # create simulation objects, and send each one an initial event message to self
    for obj_id in range(args.num_phold_objects):
        phold_obj = PholdSimulationObject(str(obj_id), args)
        simulator.add_object(phold_obj)

    # run the simulation
    simulator.initialize()
    PholdSimulationObject.record_event_header()
    event_num = simulator.simulate(args.time_max).num_events
    print("Executed {} events.\n".format(event_num))

The PHOLD model runs multiple instances of `PholdSimulationObject`.
To simplify the example, each object's name is the string representation of its integer index.
`create_and_run` creates the objects and adds them to the simulator.

Each `PholdSimulationObject` object is initialized with `args`, a namespace object that defines two attributes used by all objects:

* `args.num_phold_objects`: the number of PHOLD objects running
* `args.frac_self_events`: the fraction of events sent to self

At time 0, each PHOLD object schedules an `InitMsg` event for itself that occurs after a random exponential time delay with mean = 1.0.

The `handle_simulation_event` method handles all events.
Each event schedules one more event.
A PHOLD object uses a U(0,1) random value to randomly schedule the event for itself (with probability `args.frac_self_events`) or for another PHOLD object.

If the event is scheduled for another PHOLD object, this line obtains a reference to the object: 

    receiver = self.simulator.simulation_objects[str(obj_index)]

It uses the attribute `self.simulator`, which always references the simulation engine that is running, and `self.simulator.simulation_objects` which is a dictionary that maps simulation object names to simulation object instances.

The prediction generated by a simulation can be saved in many ways.
While illustrating more features of DE-Sim, this example simply prints them.

Each event is recorded by `record_event`.
It accesses the DE-Sim `Event` object that is passed to all event handlers.
`de_sim.event.Event` contains five useful fields:

* `sending_object`: the object that created and sent the event
* `creation_time`: the simulation time when the event was created (a.k.a. its *send time*)
* `receiving_object`: the object that received the event
* `event_time`: the simulation time when the event must execute (a.k.a. its *receive time*)
* `message`: the `SimulationMessage` carried by the event

However, rather than use the event's `event_time`, `record_event` uses `self.time` to report the simulation time when the event is being executed, as they are always equal.

In [8]:
from argparse import Namespace
args = Namespace(time_max=4,
                 frac_self_events=0.3,
                 num_phold_objects=6)
create_and_run(args)

Sender	Send	Receivr	Event	Message type
	time		time	
4	0.00	4	0.02	InitMsg
0	0.00	0	0.11	InitMsg
2	0.00	2	0.34	InitMsg
4	0.02	1	0.36	MessageSentToOtherObject
3	0.00	3	0.63	InitMsg
1	0.00	1	0.64	InitMsg
0	0.11	1	0.66	MessageSentToOtherObject
1	0.64	2	0.69	MessageSentToOtherObject
1	0.36	5	0.83	MessageSentToOtherObject
5	0.00	5	0.92	InitMsg
1	0.66	1	1.05	MessageSentToSelf
2	0.34	2	1.22	MessageSentToSelf
5	0.83	0	1.47	MessageSentToOtherObject
0	1.47	4	1.54	MessageSentToOtherObject
1	1.05	1	1.59	MessageSentToSelf
5	0.92	3	1.83	MessageSentToOtherObject
3	0.63	4	2.08	MessageSentToOtherObject
2	1.22	0	2.32	MessageSentToOtherObject
4	2.08	4	2.39	MessageSentToSelf
0	2.32	2	2.54	MessageSentToOtherObject
1	1.59	2	2.89	MessageSentToOtherObject
3	1.83	0	2.95	MessageSentToOtherObject
2	0.69	2	3.05	MessageSentToSelf
2	3.05	0	3.30	MessageSentToOtherObject
0	3.30	0	3.46	MessageSentToSelf
4	1.54	2	3.55	MessageSentToOtherObject
0	3.46	2	3.65	MessageSentToOtherObject
2	2.89	5	3.75	MessageSentToOtherObject
